## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Rikitea,PF,-23.1203,-134.9692,79.47,70,96,12.64,light rain
1,1,Canon City,US,38.4410,-105.2424,52.20,37,0,8.05,clear sky
2,2,Lerwick,GB,60.1545,-1.1494,42.80,87,87,10.29,overcast clouds
3,3,Iqaluit,CA,63.7506,-68.5145,-14.31,68,100,18.99,overcast clouds
4,4,Constitucion,CL,-35.3333,-72.4167,62.11,76,0,10.54,clear sky


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0     159
City           159
Country        159
Lat            159
Lng            159
Max Temp       159
Humidity       159
Cloudiness     159
Wind Speed     159
Description    159
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No empty rows so will create clean_df for next step 
clean_df = preferred_cities_df

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Rikitea,PF,79.47,light rain,-23.1203,-134.9692,
11,Tessalit,ML,75.43,scattered clouds,20.1986,1.0114,
13,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,
18,Takoradi,GH,79.54,scattered clouds,4.8845,-1.7554,
26,Atuona,PF,78.98,scattered clouds,-9.8000,-139.0333,
31,Puerto Ayora,EC,77.56,overcast clouds,-0.7393,-90.3518,
32,Sao Joao Da Barra,BR,76.19,clear sky,-21.6403,-41.0511,
36,Muli,MV,83.03,scattered clouds,2.9167,73.5667,
40,Georgetown,MY,87.73,few clouds,5.4112,100.3354,
43,Torit,SS,78.67,broken clouds,4.4133,32.5678,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Rikitea,PF,79.47,light rain,-23.1203,-134.9692,People ThankYou
11,Tessalit,ML,75.43,scattered clouds,20.1986,1.0114,
13,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
18,Takoradi,GH,79.54,scattered clouds,4.8845,-1.7554,Raybow International Hotel
26,Atuona,PF,78.98,scattered clouds,-9.8000,-139.0333,Villa Enata
31,Puerto Ayora,EC,77.56,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
32,Sao Joao Da Barra,BR,76.19,clear sky,-21.6403,-41.0511,Pousada Porto De Canoas
36,Muli,MV,83.03,scattered clouds,2.9167,73.5667,Medhufushi Island Resort
40,Georgetown,MY,87.73,few clouds,5.4112,100.3354,Cititel Penang
43,Torit,SS,78.67,broken clouds,4.4133,32.5678,Diplomat Guest House


In [32]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np

hotel_df = hotel_df.replace("", np.nan)
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Rikitea,PF,79.47,light rain,-23.1203,-134.9692,People ThankYou
11,Tessalit,ML,75.43,scattered clouds,20.1986,1.0114,NaN
13,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
18,Takoradi,GH,79.54,scattered clouds,4.8845,-1.7554,Raybow International Hotel
26,Atuona,PF,78.98,scattered clouds,-9.8000,-139.0333,Villa Enata


In [36]:
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Rikitea,PF,79.47,light rain,-23.1203,-134.9692,People ThankYou
13,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
18,Takoradi,GH,79.54,scattered clouds,4.8845,-1.7554,Raybow International Hotel
26,Atuona,PF,78.98,scattered clouds,-9.8000,-139.0333,Villa Enata
31,Puerto Ayora,EC,77.56,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [37]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [42]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))